## WSI图片切片

##### 1. 导入相关的包

In [ ]:
import openslide
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator

import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

##### 2. WSI切片函数

In [ ]:
def wsi_processing(img_path, save_path, tile_size):
    slide = open_slide(img_path)
    tiles = DeepZoomGenerator(slide, tile_size, overlap=0, limit_bounds=False)
    level = tiles.level_count - 1
    cols, rows = tiles.level_tiles[level]
    
    for row in range(rows):
        for col in range(cols):
            tile_name = os.path.join(save_path, '%d_%d' % (col, row))
            print("Now saving tile with title: ", tile_name)
            temp_tile = tiles.get_tile(level, (col, row))
            temp_tile_RGB = temp_tile.convert('RGB')
            temp_tile_np = np.array(temp_tile_RGB)
            plt.imsave(tile_name + ".png", temp_tile_np)

##### 3. 计算均值方差函数
上面的方法会保存大量的空白背景图片，因此可以根据自己的数据集，计算一下空白，非空白，本空白的均值和方差，以此来筛选一下图片需不需要保存。

In [ ]:
def find_mean_std_pixel_value(img_list):
    
    avg_pixel_value = []
    stddev_pixel_value= []
    for file in img_list:
        img = Image.open(file)
        image = np.array(img)
        avg = image.mean()
        std = image.std()
        avg_pixel_value.append(avg)
        stddev_pixel_value.append(std)
        
    avg_pixel_value = np.array(avg_pixel_value)  
    stddev_pixel_value=np.array(stddev_pixel_value)
        
    print("Average pixel value for all images is:", avg_pixel_value.mean())
    print("Average std dev of pixel value for all images is:", stddev_pixel_value.mean())
    
    return(avg_pixel_value, stddev_pixel_value)

In [ ]:
import glob
orig_tile_dir_name = "/Users/Desktop/Images/"

blank_img_list=(glob.glob(orig_tile_dir_name+"blank/*.png"))
partial_img_list=(glob.glob(orig_tile_dir_name+"partial/*.png"))
good_img_list=(glob.glob(orig_tile_dir_name+"good/*.png"))

blank_img_stats = find_mean_std_pixel_value(blank_img_list)
partial_img_stats = find_mean_std_pixel_value(partial_img_list)
good_img_stats = find_mean_std_pixel_value(good_img_list)

##### 4. 改进版WSI切片函数
根据前面算的数值，在保存图片前面加上一个条件。

In [ ]:
def wsi_processing(img_path, save_path, tile_size):
    slide = open_slide(img_path)
    tiles = DeepZoomGenerator(slide, tile_size, overlap=0, limit_bounds=False)
    level = tiles.level_count - 1
    cols, rows = tiles.level_tiles[level]
    
    for row in range(rows):
        for col in range(cols):
            tile_name = os.path.join(save_path, '%d_%d' % (col, row))
            # print("Now saving tile with title: ", tile_name)
            temp_tile = tiles.get_tile(level, (col, row))
            temp_tile_RGB = temp_tile.convert('RGB')
            temp_tile_np = np.array(temp_tile_RGB)
            if temp_tile_np.mean() < 230 and temp_tile_np.std() > 20:
                plt.imsave(tile_name + ".png", temp_tile_np)